In [14]:
import pandas as pd
import os
import matplotlib.pyplot as plt

# Load the uploaded file
file_path = 'inputData/morpc-qcew-2018-2022-annual-MORPC15_readable.csv'

# Load the data from the CSV file
data = pd.read_csv(file_path)


colors = [
    '#365072',  # Original
    '#34617A',  # Original
    '#0075BF',  # Original
    '#00B2BF',  # Original
    '#2A7F67',  # Original
    '#5AB65F',  # Original
    '#28556A',  # Deepened blue-gray
    '#39807D',  # Muted teal
    '#009FCC',  # Brightened blue
    '#33C1C1',  # Softened aqua
    '#3C9563',  # Rich green-teal
    '#66C47A'   # Vibrant green
]



# Set up output directory
output_dir = 'outputData/plots'
os.makedirs(output_dir, exist_ok=True)

# Filter data to exclude 'Total, all industries'
filtered_data = data[data['industry_code'] != 10]

filtered_data = filtered_data[filtered_data['own_code'] == 10]

filtered_data = filtered_data[filtered_data['agglvl_code'] == 73]

# Dictionary to store COUNTY to itemID mapping
county_item_mapping = {}

# Iterate over each county
for county in filtered_data['county_name'].unique():
    county_data = filtered_data[filtered_data['county_name'] == county]
    
    # Ensure 'YEAR', 'annual_avg_emplvl_prop', and 'industry_code_name' columns are present
    county_data = county_data[['year', 'annual_avg_emplvl_prop', 'industry_code_name']]
    
    # Aggregate to handle duplicates by taking the mean
    county_data = county_data.groupby(['year', 'industry_code_name'], as_index=False).agg({'annual_avg_emplvl_prop': 'mean'})
    
    # Pivot data to have 'industry_code_name' as columns for plotting
    county_data_pivot = county_data.pivot(index='year', columns='industry_code_name', values='annual_avg_emplvl_prop')
    
    # Normalize data for 100% stacking
    county_data_pivot_percentage = county_data_pivot.div(county_data_pivot.sum(axis=1), axis=0) * 100
    
    # Plot each county's data as a 100% stacked bar chart
    ax = county_data_pivot_percentage.plot(
        kind='bar',
        stacked=True,
        figsize=(16, 9),
        color=colors[:len(county_data_pivot_percentage.columns)],  # Use improved color palette
        title=f"{county} Employment Levels Proportions (2018-2022)"
    )
    
    plt.ylabel("Percentage of Employment Proportion")
    plt.xticks(rotation=30, fontsize=12)
    plt.xlabel("")  # Drop x-axis label
    plt.yticks(fontsize=18)
    plt.legend(title='Industry', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=18)  # Set legend font size to 18px
    plt.tight_layout()
    
    # Save the plot as a PNG file
    plot_path = os.path.join(output_dir, f"{county}_employment_levels_100pct_stacked_bar.png")
    plt.savefig(plot_path)
    plt.close()
    
    # Placeholder ID for demonstration as upload to AGO isn't performed here
    county_item_mapping[county] = f"simulated_item_id_for_{county}"

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_file_path_colored = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path_colored, index=False)

# Provide the path to the saved mappings and generated plots
output_dir, mapping_file_path_colored





('outputData/plots', 'AGO_County_ItemID_Mapping.csv')

In [15]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Directory where images are saved
output_dir = 'outputData/plots'

# Initialize dictionary to store county-item mappings
county_item_mapping = {}

# Loop through each file in the directory and upload it
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "tags": "county, population, race, demographics",
            "description": f"Population proportion over time by race for {county_name} county.",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item


        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Store item ID in the dictionary
            county_item_mapping[county_name] = f"https://morpc.maps.arcgis.com/sharing/rest/content/items/{image_item.id}/data"
            
            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the dictionary to a DataFrame and save
mapping_df = pd.DataFrame(list(county_item_mapping.items()), columns=['COUNTY', 'itemID'])
mapping_file_path = 'AGO_County_ItemID_Mapping.csv'
mapping_df.to_csv(mapping_file_path, index=False)



C:\Users\ogwynn\newvirt\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15-county region_employment_levels_100pct_stacked_bar.png with item ID: 498cb882b83849168ec5445c6525326a and set it to public


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_employment_levels_100pct_stacked_bar.png with item ID: 716d1e1e66164b37ab27760acf0f3b36 and set it to public


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

Uploaded Pickaway_employment_levels_100pct_stacked_bar.png with item ID: 3cca363841c440eeaf5f4b6627d3a6cf and set it to public


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_employment_levels_100pct_stacked_bar.png with item ID: 6362bebe17fc4266ac68a1a1d3d24e53 and set it to public


C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\newvirt\Lib\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_employment_levels_100pct_stacked_bar.png with item ID: b6afab5f41304725bdf2b965dc9adf3d and set it to public
